In [18]:
import os
import numpy as np
from skimage import io, img_as_float
from skimage.restoration import richardson_lucy
from scipy.signal import convolve2d, wiener
from skimage.util import img_as_ubyte
import cv2
from tqdm import tqdm


In [20]:
def generate_psf(kernel_size=5):
    psf = np.ones((kernel_size, kernel_size), dtype=np.float32)
    return psf / psf.sum()

In [22]:
def deblur_with_rl(image, psf, num_iter=30):
    restored_channels = []
    for c in range(3):
        restored = richardson_lucy(image[:, :, c], psf, num_iter=num_iter)
        restored_channels.append(restored)
    return np.stack(restored_channels, axis=2)

In [24]:
def deblur_with_wiener(image, psf):
    restored_channels = []
    for c in range(3):
        blurred = convolve2d(image[:, :, c], psf, mode='same', boundary='wrap')
        deblurred = wiener(blurred, mysize=None, noise=0.1)
        restored_channels.append(deblurred)
    return np.stack(restored_channels, axis=2)

In [62]:
def process_dataset(input_folder, output_folder, method='rl', psf_size=5):
    os.makedirs(output_folder, exist_ok=True)
    psf = generate_psf(psf_size)

    image_files = sorted([f for f in os.listdir(input_folder) if f.endswith(('.jpg', '.png', '.jpeg'))])

    print(f"\n🔧 {method.upper()} yöntemiyle {len(image_files)} görüntü işleniyor...\n")
    for filename in tqdm(image_files, desc=f"{method.upper()} Deblurring"):
        path = os.path.join(input_folder, filename)
        image = img_as_float(io.imread(path))
        if image.ndim != 3 or image.shape[2] != 3:
            print(f"⚠️ Renkli olmayan görüntü atlandı: {filename}")
            continue

        if method == 'rl':
            deblurred = deblur_with_rl(image, psf, num_iter=40)
        elif method == 'wiener':
            deblurred = deblur_with_wiener(image, psf)
        else:
            raise ValueError("Desteklenmeyen yöntem: 'rl' ya da 'wiener' olmalı.")

        deblurred = np.clip(deblurred, 0, 1)
        safe_image = img_as_ubyte(deblurred)
        save_path = os.path.join(output_folder, f"{method}_{filename}")
        io.imsave(save_path, safe_image)

In [64]:
blurType = ['Defocus Blurred','Bilateral Filtered','Gaussian Blurred','Low Light Blurred','Motion Blurred' ]
for file in blurType:
    process_dataset(
        input_folder="datasets/newBlurImages/" + file + "/images",
        output_folder="datasets/deblurRL/" + file + "/images",
        method='rl',
        psf_size=7
    )


🔧 RL yöntemiyle 1000 görüntü işleniyor...



RL Deblurring: 100%|████████████████████████| 1000/1000 [19:32<00:00,  1.17s/it]


In [20]:
blurType = ['Bilateral Filtered','Defocus Blurred','Gaussian Blurred','Low Light Blurred','Motion Blurred' ]
for file in blurType:
    process_dataset(
        input_folder="datasets/newBlurImages/" + file,
        output_folder="datasets/deblurWiener/" + file,
        method='wiener',
        psf_size=7
    )


🔧 WIENER yöntemiyle 1000 görüntü işleniyor...



WIENER Deblurring: 100%|████████████████████| 1000/1000 [02:02<00:00,  8.17it/s]



🔧 WIENER yöntemiyle 1000 görüntü işleniyor...



WIENER Deblurring: 100%|████████████████████| 1000/1000 [02:02<00:00,  8.16it/s]



🔧 WIENER yöntemiyle 1000 görüntü işleniyor...



WIENER Deblurring: 100%|████████████████████| 1000/1000 [02:07<00:00,  7.84it/s]



🔧 WIENER yöntemiyle 1000 görüntü işleniyor...



WIENER Deblurring: 100%|████████████████████| 1000/1000 [02:07<00:00,  7.85it/s]



🔧 WIENER yöntemiyle 1000 görüntü işleniyor...



WIENER Deblurring: 100%|████████████████████| 1000/1000 [01:59<00:00,  8.34it/s]
